# Quickstart Guide for Strands Agents

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.

## Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Basic understanding of Python programming

Lets get started !

In [1]:
# Install Strands using pip

!pip install strands-agents strands-agents-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


## Creating Your Financial Advisor Agent

Lets put in our AWS credentials, and ensure that it works!


### Model Provider

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.5 Sonnet in the region of your current AWS environment

You can specify a different model in Amazon Bedrock providing the model ID string directly:

In [3]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)

{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0'}



For more control over the model configuration, you can create a `BedrockModel` provider instance:

In [4]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-west-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)


More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


## Lets Build a Task-Specific Agent - Financial Advisor Bot 🍽️

Lets create a practical example of a task-specific agent. We create a `FinBot` that gives financial advice. Lets dive in !!

Here's what we will create :

In [5]:
# Install the required packages
%pip install duckduckgo-search # Also install strands-agents strands-agents-tools if you haven't already

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.5 MB/s eta 0:00:00


In [6]:
from strands import Agent, tool
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import RatelimitException, DuckDuckGoSearchException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [7]:
# Create a recipe assistant agent
recipe_agent = Agent(
    system_prompt="""You are a FinBot, a financial advising assistant.
    Help users by giving financial advice based on age, income, financial goals, risk tolerance, and time horizon.
    Use the websearch tool to go to yahoo finance and tell me which stock or share is worth investing in like S&P 500 or S&P 50.
    Give me an explanation why you pick that specific one over the others or whichever one you are comparing with.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [9]:
response = recipe_agent("I am 25 yrs old, make $15,000, plan to buy a car in the future, also plan to save money for my retirement, and have an emergency fund.")

print(response)

Thank you for providing more details about your financial situation. It's great to hear that you already have an emergency fund in place - that's an excellent foundation for your financial planning.

Given your age (25), income ($15,000), goals of buying a car and saving for retirement, and the fact that you've already established an emergency fund, let me search for the most current information on suitable investment options for you.
Tool #4: websearch

Tool #5: websearch

Tool #6: websearch

Tool #7: websearch
Based on the research I was able to gather and your financial situation, here's my investment recommendation:

# Investment Recommendation for Your Dual Goals

## Recommended Investment: Vanguard S&P 500 ETF (VOO)

After analyzing current market options, I recommend the Vanguard S&P 500 ETF (VOO) as the best investment option for your situation. Here's why I'm recommending this over other alternatives:

### Why VOO is Best for Your Situation:

1. **Ultra-low expense ratio (0.03

#### And that's it! We now have a running usecase agent with tools in just a few lines of code 🥳.

For more detailed quickstart guide, check out the [Strands documentation](https://strandsagents.com/0.1.x/user-guide/quickstart/).